## How to trace/convert Transformer model into Triton acceptable models?
- source https://github.com/sachinsharma9780/AI-Enterprise-Workshop-Building-ML-Pipelines/blob/main/

In [ ]:
!pip3 install torch torchvision torchaudio --quiet

In [ ]:
import torch
from torch.nn import functional as F

In [2]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

sentence = "Hello, my dog is cute!"
labels = "happy"


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
input_ids = tokenizer.encode(sentence, hypothesis, return_tensors = 'pt', max_length= 256, truncation = True, 
                            padding = "max_length")

mask = input_ids !=1
mask = mask.long()

## Tracing PyTorch Model with torchscript
- this is to create serializable and optimizable models, in equivalent to convert a tensorflow model with TensorRT

In [23]:
class PyTorch_to_TorchScript(torch.nn.Module):
    def __init__(self):
        super(PyTorch_to_TorchScript, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli')
    def forward(self, data, attention_mask=None):
        return self.model(data, attention_mask)[0]

In [24]:
# after trace it will save the model in cwd
pt_model = PyTorch_to_TorchScript().eval()

traced_script_module = torch.jit.trace(pt_model, (input_ids, attention_mask), strict=False)
traced_script_module.save("./model.pt")

## Save the converted model to the Triton model repository folder

In [26]:
import shutil
import os
folder_name = 'sentence_bert'
# os.mkdir(f'/root/model_repository/{folder_name}')
# os.mkdir(f'/root/model_repository/{folder_name}/1')
shutil.copy('model.pt', f'/root/model_repository/{folder_name}/1')

'/root/model_repository/sentence_bert/1/model.pt'